# SMC sampler PRC-ABC algorithm (Peters et al., 2009)

Dans cette partie nous avons essayé de mettre en place le sampler décrit dans l'Annexe A de notre article. Il est basé sur à la fois sur la technique du Sequential Monte Carlo et sur la Partial Rejection.  

Remarque : Comme nous avons pas obtenus de résultats satisfaisants ce code contient volontairement quelques "print" supplémentaires pour que le correcteur puisse identifier plus facilement 
ce qui marche ou pas.

Tout le code est rapide à exécuter sauf la cellule qui fait tourner l’algorithme, elle sera indiquée comme telle. 

In [1]:
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal

On va maintenant définir des fonctions qui nous seront utiles par la suite:

In [2]:
def prior_sample():
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    alpha = uniform.rvs(1.1, 0.9)  # U[1.1, 2]
    beta = uniform.rvs(-1, 2)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 600)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [3]:
def univ_alpha_stable_sampler(params,size,seed) :
    """
    Applies the sampling algoritm from the article for alpha-stable distributions

    Args:
        params (list): parameters of the alpha stable distribution (alpha,beta,gamma,delta)
        size (int): sample size
        seed (int): for reproducibility

    Returns:
        np.array : n samples from the alpha stable 
    """
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [4]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [5]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [6]:
def psi(y, x, covariance = np.diag([0.25,0.25,1,1])):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=covariance)


In [7]:
def cov_estimate(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(params = theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [8]:
def norm2(u) :
    # compute the norm 2 of a vector
    return np.sqrt(np.sum(u*u))


In [9]:
def K(u, sigma) :
    # Define a Gaussian Kernel in the dimension of S(.)
    d = 4
    return np.exp(-(u.T@np.linalg.inv(sigma)@u)/2)/(np.sqrt(2*np.pi)**d*np.sqrt(np.linalg.det(sigma)))




In [10]:
def resample(theta, weights,N) :
    # to resample with respect to the weights
    
    normalized_weights = weights['t']/np.sum(weights['t'])
    print(theta['t-1'].shape)
    new_indices = np.random.choice(range(N),replace=True, p=normalized_weights)
    theta['t'] = theta['t'][new_indices]

    weights['t'] = np.ones(N)/N

In [11]:
def sample_from_M_t(N, theta_prev, weights_prev):
    """sample theta_i from M_t(theta_i)

    Args:
        N (int): nb of particles
        theta_prev (np.array of size N): thetas rom the previous time step
        weights_prev (np.array of size N): weights from the previous time step

    Returns:
        a theta [alpha, beta, gamma, delta]
    """

    normalized_weights = weights_prev/np.sum(weights_prev)
    # Choose a component based on the weights
    component = np.random.choice(range(N), p=normalized_weights)
    # Generate a sample from the chosen component
    sample = np.random.multivariate_normal(mean = theta_prev[component], cov = np.diag([0.25,0.25,1,1]))
    
    return sample 

In [12]:
def M_t(N,theta_i,theta_prev, weights_prev) :
    # defines the mutation kernel M_t as the weighted sum of densities
    
    densities = np.array([ psi(theta_i, theta_prev[i]) for i in range(N)])
    return np.sum(weights_prev*densities)
    

In [13]:
def normalize(u) :
    # normalize a vector u (sum of coordinates equals 1)
    return u/np.sum(u)


In [14]:
def prior_density(theta_i) :
    # return the product of the uniform prior densities evaluated respectively in alpha, beta, gamma, delta

    alpha,beta,gamma,delta = theta_i[0],theta_i[1],theta_i[2],theta_i[3]
    
    return uniform.pdf(alpha,1.1,0.9)*uniform.pdf(beta,-1,2)*uniform.pdf(gamma,0,300)*uniform.pdf(delta,-300,600)

In [15]:
def initialization(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=124) :
    """ Initialize the algorithm  

    Args:
        T (int, optional): number of time steps. Defaults to 10.
        N (int, optional): number of particles. Defaults to 1000.
        n (int, optional): length of y and x. Defaults to 200.
        covariance (np.array, optional): lambda in the paper. Defaults to np.diag([0.25, 0.25, 1, 1]).
        seed (int, optional): to init the RNG for reproducibility. Defaults to 124.

    Returns:    y ( np.array of size n) : "observed" data.
                x (np.array of size n) : auxiliary data simulated from theta_i.
                theta (dict) : stores the N thetas at time t and t-1 with (2 keys).
                weights (dict) : stores the N weigths at time t and t-1 with (2 keys).
                sigma_hat (4x4 np.array) : covariance matrix of S(x) 
        
    """
    # Tolerance schedule
    epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  
    epsilon = epsilons[0]
    
    # generate the "observed data" y :
    levy_stable.parameterization = 'S0'
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    
    # estimate sigma_hat :
    sigma_hat = cov_estimate(np.array([1.7,0.9,10,10]))
    print("sigma_hat :\n", sigma_hat)
    print('det(sigma_hat) = ', np.linalg.det(sigma_hat))
    
    # Sample theta from the prior : 
        # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample() for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    print('theta["t"] = ',np.round(theta['t'][:5],4))
    
   # generate the x 
    x = np.array([levy_stable.rvs(theta['t'][i][0], theta['t'][i][1], loc=theta['t'][i][3], scale=theta['t'][i][2], size=200) for i in range(N)])
    print('x.shape = ',x.shape) 
    
    # set weights their initial values :
    initial_weights = np.array([K((S(y)-S(x[i]))/epsilon,(epsilon**2)*sigma_hat)/epsilon for i in range(N)])
    print("weights = ", initial_weights)

    weights = {"t" : initial_weights, "t-1" : initial_weights}
    
    return y,x,theta,weights,sigma_hat

### Remarque :  
Il n'est pas précisé dans l'article si les poids $ W^{i}_t(\theta^{i}_t) $ doivent sommer à 1 pour $i = 1...N$. Donc on ne l'a pas fait...


Initialisation des paramètres :

In [16]:
arg = initialization()

sigma_hat :
 [[ 2.53860936e+00 -1.22821067e-01 -1.30866612e-01 -6.99933804e+00]
 [-1.22821067e-01  2.26316622e-02 -2.04722122e-02  1.00018299e+00]
 [-1.30866612e-01 -2.04722122e-02  2.60621022e-01 -2.17568715e+00]
 [-6.99933804e+00  1.00018299e+00 -2.17568715e+00  8.15704477e+02]]
det(sigma_hat) =  6.787464192209966
theta["t"] =  [[ 1.536400e+00  3.197000e-01  6.602770e+01  1.639823e+02]
 [ 1.935200e+00  1.326000e-01  1.184304e+02  5.319600e+01]
 [ 1.880000e+00 -1.671000e-01  1.009823e+02  2.194340e+01]
 [ 1.572100e+00  1.434000e-01  1.823109e+02  3.775380e+01]
 [ 1.603900e+00  6.450000e-01  7.207700e+00 -2.047916e+02]]
x.shape =  (1000, 200)
weights =  [9.72269109e-18 9.72269109e-18 9.72269045e-18 9.72269102e-18
 9.72269109e-18 9.72269109e-18 9.72269109e-18 9.72269109e-18
 9.72269109e-18 9.72269109e-18 9.72269109e-18 9.72269109e-18
 9.72269109e-18 9.72269109e-18 9.72269109e-18 9.72269109e-18
 9.72269109e-18 9.72269109e-18 9.72269109e-18 9.72269109e-18
 9.72269109e-18 9.72269109e-18 9.

Nous arrivons maintenant à l'application de l'algorithme SMC en lui même : 

### Atention la cellule suivante prend du temps à s'éxécuter.

In [17]:
N =1000
n=200
covariance = np.diag([0.25, 0.25, 1, 1])
seed=124

# Tolerance schedule
epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  
T = len(epsilons)
levy_stable.parameterization = 'S0'

# get the values of y,x,theta,weights,sigma_hat from the initialisation function :
y,x,theta,weights,sigma_hat = arg

#set t = 2
t=2

# to store the theta['t'] and the weights['t'] for t = 1...T :
res = [[theta['t'],weights['t']]]

while t<T :
    print("t = ",t)

    # update the tolerance parameter epsilon_t : 
    epsilon = epsilons[t]
    print("epsilon = ",epsilon)
    
    # store the values of the thetas and the weights to update theta['t-1] and weights['t-1] later :
    theta_maj = theta['t']
    weights_maj = weights['t']
    
    i = 0 
    
    # dynamic thresehold : 
    c_t = np.quantile(weights['t'], 0.9) 

    while i<N : 
        print('###########    i = ',i)

        #----------Mutation and correction : 
        
        # sample theta_i from M_t(theta_i) :
        theta['t'][i] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
        
        # sometimes the values sampled for one of the 4 component of theta_i is out of the prior
        # so we draw another thetha_i : 
        if prior_density(theta['t'][i])==0:
            print("the prior is O")
            continue
        # Draw x_i from p(x_i/theta_i) :
        x[i] = levy_stable.rvs(theta['t'][i][0], theta['t'][i][1], loc=theta['t'][i][3], scale=theta['t'][i][2], size=200)
        print(f'theta["t"][{i}]= ',theta['t'][i])

        # Update the weight of theta_i :
        weights['t'][i] = (K((S(y)-S(x[i]))/epsilon,(epsilon**2)*sigma_hat)/epsilon)*prior_density(theta['t'][i])/M_t(N,theta['t'][i],theta['t-1'],weights['t-1'])
        
        # Draw uniformly a random value between 0 and 1 :

        u = np.random.uniform(low=0, high=1)

        print("rapport W_i/c_t = ", weights['t'][i]/c_t)
        
        # compute the probability for theta_i to be rejected :
        proba = 1-np.minimum(1,weights['t']/c_t)
        
        print("c_t = ", c_t)
        print(f"weights['t'][{i}] = ", weights['t'][i])
        print('sum(weights) = ', np.sum(weights['t']))
        print(f'proba[{i}] = ', proba[i])

        if u<=proba[i] :
            #---Rejection : 
            print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} rejected !")
            continue
        else : 
            #---Acceptance
            print(f"theta['t'][{i}] = {np.round(theta['t'][i],3)} accepted !")
            # update the weights :
            weights['t'][i] = weights['t'][i]/(1-proba[i])
            # go to the next theta_i
            i = i+1
    
    t+=1
    # update the values 
    theta['t-1'],weights['t-1'] = theta_maj,weights_maj
    # update the results
    res.append((theta['t'], weights['t']))

    # We didn't understood well the resampling part, but according to "On sequential Monte Carlo, partial rejection control and approximate Bayesian computation"
    # G. W. Peters∗† Y. Fan∗ S. A. Sisson∗‡ November 11, 2009, this step is not always necessary...so didn't implement it truly
    """
    # Resample : 
    weights['t'] = np.ones(N)/N
    weights['t-1'] = np.ones(N)/N
    """


t =  2
epsilon =  800
###########    i =  0
the prior is O
###########    i =  0
the prior is O
###########    i =  0
the prior is O
###########    i =  0
the prior is O
###########    i =  0
the prior is O
###########    i =  0
theta["t"][0]=  [   1.41853568   -0.41939725  258.53871005 -283.98570557]
rapport W_i/c_t =  6208891733799.296
c_t =  9.722691090319275e-18
weights['t'][0] =  6.036713634096741e-05
sum(weights) =  6.036713635068039e-05
proba[0] =  0.0
theta['t'][0] = [   1.419   -0.419  258.539 -283.986] accepted !
###########    i =  1
the prior is O
###########    i =  1
theta["t"][1]=  [   1.38888102   -0.29351317  256.68270957 -284.30619862]


rapport W_i/c_t =  9.383226057365928
c_t =  9.722691090319275e-18
weights['t'][1] =  9.123020838640335e-17
sum(weights) =  6.03671363507619e-05
proba[1] =  0.0
theta['t'][1] = [   1.389   -0.294  256.683 -284.306] accepted !
###########    i =  2
theta["t"][2]=  [   1.57885716   -0.6250183   258.88922619 -283.59285124]
rapport W_i/c_t =  2.0264965360163933
c_t =  9.722691090319275e-18
weights['t'][2] =  1.970299981528946e-17
sum(weights) =  6.036713635077188e-05
proba[2] =  0.0
theta['t'][2] = [   1.579   -0.625  258.889 -283.593] accepted !
###########    i =  3
theta["t"][3]=  [ 1.79492558e+00 -2.01585510e-01  2.58727840e+02 -2.85898459e+02]
rapport W_i/c_t =  14.25581247279638
c_t =  9.722691090319275e-18
weights['t'][3] =  1.3860486091451975e-16
sum(weights) =  6.0367136350900765e-05
proba[3] =  0.0
theta['t'][3] = [ 1.79500e+00 -2.02000e-01  2.58728e+02 -2.85898e+02] accepted !
###########    i =  4
the prior is O
###########    i =  4
theta["t"][4]=  [   1.31412299   -0.28988079 

KeyboardInterrupt: 

Nous avons rencontrer plusieurs difficultés qui pourraient être la cause de résultats non optimaux de notre implémentation du SMC sampler :  
- nous n'étions pas sûr que la forme exacte du noyau K 
- de même la forme exacte de $\pi_{LF}(\theta|y)$ n'était pas clair, et cela malgré la consultation de l'article :On sequential Monte Carlo, partial rejection control and approximate Bayesian computation" de G. W. Peters, Y. Fan et S. A. Sisson (November 11, 2009) où ils disent que l'on peut en calculer la forme exacte. Comme ils disent que P = 1 dans l'estimation de 
$\pi_{LF}(\theta|y)$ par une somme de 1 à P nous avons donc dit que $\pi_{LF}(\theta|y)\propto \pi(x)\pi(y|x,\theta) $
- autre source possible d'erreur/ralentissement du code : les valeurs prisent par $K_{\epsilon}(\frac{S(y)-S(y)}{\epsilon})/\epsilon$ sont très petites, typiquement $10^{-15}$ par exemple, cela multiplie sans doutes les approximations et autres erreurs de calcul.
- enfin l'étape de resampling déjà mentionnée plus haut, ce qui nous a paru étranges c'est qu'il faut re-sampler (sans remises) N valeurs  en accord avec les poids mais dans une liste de N valeurs, donc on reprend juste toute la liste... Et puis quand bien même ce serait un tirage avec remise, mettre les poids à 1/N entraîne que le $\theta^0_i$ est rejeté à chaque fois car sont poids est trop faible du fait de la def de $\pi_{LF}(\theta|y)$ retenue.